# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [27]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [7]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [13]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [18]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [64]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


<h2>1. 10 countries with the lowest infant mortality rates </h2>

In [78]:
# Create DataFrame and save countries and mortality rates
df_infm = pd.DataFrame(columns=['country','infant_mort'])
count = 0
for element in document.iterfind('country'):
    df_infm.set_value(index=count,col='country',value=element.find('name').text)
    for subelement in element.getiterator('infant_mortality'):
        df_infm.set_value(index=count,col='infant_mort',value=subelement.text)
    count+=1


In [79]:
df_infm['infant_mort'] = pd.to_numeric(df_infm['infant_mort'])
df_infm = df_infm.dropna()
df_infm.sort_values(by='infant_mort',ascending=True).head(10)

,country,infant_mort
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


<h2>2. 10 cities with the largest population</h2>

In [102]:
# Create DataFrame and save countries and population (of the last recorded population. ie. assumes the last in the tree is the most recent pop)
df_pop = pd.DataFrame(columns=['country','pop'])
count = 0
for element in document.iterfind('country'):
    country = element.find('name').text

    df_pop.set_value(index=count,col='country',value=country)
    
    for subelement in element.findall('population'):
        pop = subelement.text

    #write to dataframe the last of the population tags (saved in pop) which is the most recent
    df_pop.set_value(index=count,col='pop',value=pop)   

    count+=1


In [103]:
df_pop['pop'] = pd.to_numeric(df_pop['pop'])

In [104]:
df_pop.sort_values(by='pop',ascending=False).head(10)

,country,pop
55,China,1360720000
67,India,1210854977
120,United States,318857056
88,Indonesia,252124458
176,Brazil,202768562
57,Pakistan,173149306
202,Nigeria,164294516
65,Bangladesh,149772364
23,Russia,143666931
98,Japan,127298000


<h2>3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)</h2>

In [152]:
# Create DataFrame and save countries and population (of the last recorded population. ie. assumes the last in the tree is the most recent pop)
df_eth = pd.DataFrame(columns=['country','ethnic_group','grp_pop'])
count = 0
for element in document.iterfind('country'):
    country = element.find('name').text
    
    for subelement in element.findall('population'):
        pop = subelement.text
    
    #write to dataframe the last of the population tags (saved in pop) which is the most recent
    df_pop.set_value(index=count,col='pop',value=pop)   

    for e_subel in element.findall('ethnicgroup'):
        ethnic_group = (e_subel.text)
        ethnic_group_pop = (float(list(e_subel.attrib.values())[0]))/100*df_pop['pop'][df_pop['country']==country]
        ethnic_group_pop = int(ethnic_group_pop)
        df_eth.set_value(index=count,col='country',value=country)
        df_eth.set_value(index=count,col='ethnic_group',value= ethnic_group)
        df_eth.set_value(index=count,col='grp_pop',value= ethnic_group_pop)

    count+=1


In [153]:
df_pop['pop'] = pd.to_numeric(df_pop['pop'])

In [154]:
df_eth=df_eth.fillna(0)

In [157]:
grpd_pop = df_eth.groupby('ethnic_group').sum()

In [161]:
grpd_pop.sort_values(by='grp_pop',ascending=False).head(10)

,grp_pop
ethnic_group,
Han Chinese,1245058800
African,248873336
Bengali,146776916
Japanese,126534212
Malay,108100168
Eastern Hamitic,82830376
Arab-Berber,80060796
European,73515162
Thai,49486244


<h2>4. Name and country of a) longest river, b) largest lake and c) airport at highest elevation</h2>

In [214]:
df_locs = pd.DataFrame(columns=['type','name','val','country'])
#type can be : river, lake, airport
#  val is the measure (length or elevation)
count = 0
#geos = ['river']
geos = ['river','airport','lake']
for geo in geos:
    print (document.iterfind(geo))
    print (geo)
    for element in document.iterfind(geo):
        name = element.find('name').text
        if geo == 'airport':
            try:
                val = element.iterfind('elevation').text
            except:
                val = 0
            val = (element.find('elevation').text)
        elif geo == 'river':
            try:
                val = element.iterfind('length').text
            except:
                val = 0
        elif geo == 'lake':
            try:
                val = (element.find('area').text)
            except:
                val = 0
        typ = geo
        country = element.attrib['country']
        
        df_locs.set_value(index=count,col='country',value=country)
        df_locs.set_value(index=count,col='val',value= val)
        df_locs.set_value(index=count,col='type',value= geo)
        df_locs.set_value(index=count,col='name',value= name)
        count+=1   


<generator object prepare_child.<locals>.select at 0x000000000D320D00>
river
<generator object prepare_child.<locals>.select at 0x000000000D320D00>
airport
<generator object prepare_child.<locals>.select at 0x000000000D320D00>
lake


In [215]:
df_locs['val'] = pd.to_numeric(df_locs['val'])

In [216]:
top10_lakes = df_locs[df_locs['type']=='lake'].sort_values(by='val',ascending=False).head(10)
top10_lakes

,type,name,val,country
1607,lake,Caspian Sea,386400.0,R AZ KAZ IR TM
1662,lake,Lake Superior,82103.0,CDN USA
1634,lake,Lake Victoria,68870.0,EAT EAK EAU
1659,lake,Lake Huron,59600.0,CDN USA
1661,lake,Lake Michigan,57800.0,USA
1600,lake,Dead Sea,41650.0,IL JOR WEST
1636,lake,Lake Tanganjika,32893.0,ZRE Z BI EAT
1651,lake,Great Bear Lake,31792.0,CDN
1596,lake,Ozero Baikal,31492.0,R
1642,lake,Lake Malawi,29600.0,MW MOC EAT


In [217]:
top10_airports = df_locs[df_locs['type']=='airport'].sort_values(by='val',ascending=False).head(10)
top10_airports 

,type,name,val,country
318,airport,El Alto Intl,4063.0,BOL
457,airport,Lhasa-Gonggar,4005.0,CN
479,airport,Yushu Batang,3963.0,CN
1051,airport,Juliaca,3827.0,PE
1053,airport,Teniente Alejandro Velasco Astete Intl,3311.0,PE
320,airport,Juana Azurduy De Padilla,2905.0,BOL
572,airport,Mariscal Sucre Intl,2813.0,EC
1043,airport,Coronel Fap Alfredo Mendivil Duarte,2719.0,PE
1045,airport,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0,PE
930,airport,Licenciado Adolfo Lopez Mateos Intl,2581.0,MEX


In [209]:
top10_rivers = df_locs[df_locs['type']=='river'].sort_values(by='val',ascending=False).head(10)
top10_rivers

,type,name,val,country
0,river,Thjorsa,0,IS
163,river,Allegheny River,0,USA
151,river,Detroit River,0,CDN USA
152,river,Saint Marys River,0,CDN USA
153,river,Straits of Mackinac,0,USA
154,river,Manicouagan,0,CDN
155,river,Riviere Richelieu,0,CDN
156,river,Merrimack River,0,USA
157,river,Connecticut River,0,USA
158,river,Hudson River,0,USA


In [ ]:
<airport iatacode="KBL" city="cty-Afghanistan-Kabul" country="AFG">
      <name>Kabul Intl</name>
      <latitude>34.565853</latitude>
      <longitude>69.212328</longitude>
      <elevation>1792</elevation>
      <gmtOffset>5</gmtOffset>
   </airport>

   <river id="river-Thjorsa" country="IS">
      <name>Thjorsa</name>
      <to watertype="sea" water="sea-Atlantic"/>
      <area>7530</area>
      <length>230</length>
      <source country="IS">
         <latitude>65</latitude>
         <longitude>-18</longitude>
      </source>
      <estuary country="IS">
         <latitude>63.9</latitude>
         <longitude>-20.8</longitude>
      </estuary>
   </river>

   <lake id="lake-Inarisee" country="SF">
      <name>Inari</name>
      <located country="SF" province="lteil-LAP-SF"/>
      <to watertype="river" water="river-Paatsjoki"/>
      <area>1040</area>
      <latitude>68.95</latitude>
      <longitude>27.7</longitude>
      <elevation>119</elevation>
      <depth>92</depth>
   </lake>